#########################
# Make sure you are using the `conda_pytorch_p36` Jupyter Kernel for SageMaker!
#########################

# PyTorch Training and Serving in SageMaker "Script Mode"

Script mode is a training script format for PyTorch that lets you execute any PyTorch training script in SageMaker with minimal modification. The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) handles transferring your script to a SageMaker training instance. On the training instance, SageMaker's native PyTorch support sets up training-related environment variables and executes your training script. In this tutorial, we use the SageMaker Python SDK to launch a training job and deploy the trained model.

Script mode supports training with a Python script, a Python module, or a shell script. In this example, we use a Python script to train a classification model on the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). In this example, we will show how easily you can train a SageMaker using PyTorch scripts with SageMaker Python SDK. In addition, this notebook demonstrates how to perform real time inference with the [SageMaker PyTorch Serving container](https://github.com/aws/sagemaker-pytorch-serving-container). The PyTorch Serving container is the default inference method for script mode. For full documentation on deploying PyTorch models, please visit [here](https://github.com/aws/sagemaker-python-sdk/blob/master/doc/using_pytorch.rst#deploy-pytorch-models).

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

## Install SageMaker Python SDK with TensorFlow 2.x Support (>1.49)

In [2]:
!pip install sagemaker --upgrade --ignore-installed --no-cache

    100% |████████████████████████████████| 296kB 65.6MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 58.6MB/s ta 0:00:01
    100% |████████████████████████████████| 20.1MB 104.5MB/s ta 0:00:01
    100% |████████████████████████████████| 1.3MB 116.9MB/s ta 0:00:01
    100% |████████████████████████████████| 26.1MB 119.9MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 72.4MB/s ta 0:00:01
    100% |████████████████████████████████| 5.9MB 111.0MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 79.9MB/s ta 0:00:01
    100% |████████████████████████████████| 583kB 111.6MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 116.9MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 80.4MB/s ta 0:00:01
    100% |████████████████████████████████| 122kB 98.5MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 116.5MB/s ta 0:00:01
    100% |████████████████████████████████| 552kB 116.4MB/s ta 0:00:01
    100% |███

In [3]:
!pip list

Package                            Version        
---------------------------------- ---------------
alabaster                          0.7.10         
anaconda-client                    1.6.14         
anaconda-project                   0.8.2          
asn1crypto                         0.24.0         
astroid                            1.6.3          
astropy                            3.0.2          
attrs                              18.1.0         
autovizwidget                      0.13.1         
awscli                             1.16.283       
Babel                              2.5.3          
backcall                           0.1.0          
backports.shutil-get-terminal-size 1.0.0          
bcrypt                             3.1.7          
beautifulsoup4                     4.6.0          
bitarray                           0.8.1          
bkcharts                           0.2            
blaze                              0.11.3         
bleach                         

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
!pip uninstall numpy

## Create the SageMaker Session

In [4]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

ImportError: Something is wrong with the numpy installation. While importing we detected an older version of numpy in ['/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/numpy']. One method of fixing this is to repeatedly uninstall numpy until none is found, then reinstall this version.

## Setup the Service Execution Role and Region
Get IAM role arn used to give training and hosting access to your data.  See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).

In [ ]:
role = get_execution_role()
print('RoleARN:  {}\n'.format(role))

region = sagemaker_session.boto_session.region_name
print('Region:  {}'.format(region))

## Training Data

### Copy the Training Data to Your Notebook Disk

In [ ]:
local_data_path = './data'

In [ ]:
from torchvision import datasets, transforms

datasets.MNIST(local_data_path, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

There are four ``.npy`` file under this prefix:
* ``train_data.npy``
* ``eval_data.npy``
* ``train_labels.npy``
* ``eval_labels.npy``

In [ ]:
!ls -R {local_data_path}

### Upload the Data to S3 for Distributed Training Across Many Workers
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.

This is S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.

In [ ]:
bucket = sagemaker_session.default_bucket()
data_prefix = 'sagemaker/pytorch-mnist/data'

In [ ]:
training_data_uri = sagemaker_session.upload_data(path=local_data_path, bucket=bucket, key_prefix=data_prefix)
print('input spec (in this case, just an S3 path): {}'.format(training_data_uri))

In [ ]:
!aws s3 ls --recursive {training_data_uri}

## Train
### Training Script
The `pytorch_mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [ ]:
!ls ./src/mnist_pytorch.py

You can add custom Python modules to the `src/requirements.txt` file.  They will automatically be installed - and made available to your training script.

In [ ]:
!cat ./src/requirements.txt

### Train with SageMaker `PyTorch` Estimator

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure.  We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters.  In this case we are going to run our training job on two(2) `ml.p3.2xlarge` instances.  Alternatively, you can specify `ml.c4.xlarge` instances.  This example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)).  The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.

In [ ]:
from sagemaker.pytorch import PyTorch

mnist_estimator = PyTorch(entry_point='mnist_pytorch.py',
                          source_dir='./src',
                          role=role,
                          framework_version='1.3.1',
                          train_instance_count=2,
                          train_instance_type='ml.p3.2xlarge',
                          hyperparameters={
                            'epochs': 6,
                            'backend': 'gloo'
                          })

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem of each worker, so our training script can simply read the data from disk.

## `fit` the Model (Approx. 15 mins)

To start a training job, we call `estimator.fit(training_data_uri)`.

In [ ]:
mnist_estimator.fit(inputs={'training': inputs}, wait=False)

training_job_name = mnist_estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

In [ ]:
mnist_estimator = PyTorch.attach(training_job_name=training_job_name)

# Option 1:  Perform Batch Predictions Directly in the Notebook

In [ ]:
# TODO

# Option 2:  Perform Batch Predictions with a Batch Transform Job

In [ ]:
batch_output_uri = '{}/batch_output'.format(training_data_uri) # The location to store the results

mnist_transformer = mnist_estimator.transformer(instance_count=1, instance_type='ml.p3.2xlarge', output_path=batch_output_uri)

TODO:  We may need to use a custom data transformer to convert numpy format into json

In [ ]:
mnist_transformer.transform(data=training_data_uri, data_type='S3Prefix') # , content_type='text/json', split_type='Line')

In [ ]:
!aws --region {region} s3 ls --recursive {batch_output_uri}

# Option 3:  Create a SageMaker Endpoint and Perform REST-based Predictions

### Deploy the Trained Model to a SageMaker Endpoint (Approx. 10 mins)

After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `pytorch_mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`. Here we will deploy the model to a single `ml.p3.2xlarge` instance.  Alternatively, you can use a `ml.c5.2xlarge` instance.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge')

### Invoke the Endpoint

We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a `data` variable in this notebook, which we can then pass to the `predictor`.

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

The value of `data` is retrieved from the HTML above.

In [ ]:
print(data)

In [ ]:
import numpy as np

image = np.array([data], dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [ ]:
# mnist_estimator.delete_endpoint()

sagemaker.Session().delete_endpoint(predictor.endpoint)